# Comp Econ Homework Set 5

**Author**: Chase Coleman

**NYU ID**: N10879183

**Due Date**: 7 March 2016

## Question

Repeat exactly the exercise in homework set 4, but now in Julia. To estimate the bias at each $(\alpha, n)$ pair, take the average over 10,000 simulations.

Using `IJulia`, include your code in a Jupyter notebook that displays the resulting figure int he browser, as well as the time elapsed. In measuring the time elapsed, include only the time it takes to generate all the data. You don't need to include the time it takes to generate the figure.

Try to optimize your code.

## Answer

We will take the same approach as in homework 4 and not store any of the simulations by computing the regression coefficient as the simulation is run. This prevents the majority of memory allocation. The regression equation we can use for one dimensional regression is

$$\hat{\alpha} = \frac{\sum_i x_i y_i - E[x] E[y]}{\sum x_i^2 - n E[x]^2}$$

We are supposed to compute the bias for each value of $T = \{50, 100, 150, \dots, 500\}$. If we are smart about our algorithm, we can actually get away with only doing the simulation for the 500 periods and storing the intermediate regressions as we go.

The takeaway from this notebook is that the best way to speed up code is to build a better mouse trap.

In [2]:
using Benchmarks
using PlotlyJS

We will leverage Julia's parallel computing, so we will start 4 additional processors on our computer.

In [3]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

First we will create a simple AR(1) type which will allow us to simulate cleanly. We will also write a function which takes $\sum_{i=1}^t x_i y_i$, $\sum_{i=1}^t x_i$, $\sum_{i=1}^t y_i$, $\sum_{i=1}^t x_i^2$, and $t$ as inputs and returns the regression coefficient as presented above.

In [4]:
@everywhere begin

immutable AR1
    α::Float64
    β::Float64
    σ::Float64
end

@inline Base.step(ar::AR1, x::Float64) = ar.β + ar.α*x + ar.σ*randn()
@inline Base.step(ar::AR1, x::Vector{Float64}, n) =
    ar.β + ar.α*x + ar.σ*randn(n)

@inline comp_coeff(xiyi, xi, yi, xixi, t) = 
    (xiyi - (1/t)*xi*yi) / (xixi - (1/t)*xi*xi)
end

We then set all of the parameter values that we will need

In [12]:
αvals = [0.6, 0.7, 0.8, 0.9]
Tvals = 50:50:500
β, σ, N = 0.0, 0.1, 10000

myar1 = [AR1(α, β, σ) for α in αvals]

4-element Array{AR1,1}:
 AR1(0.6,0.0,0.1)
 AR1(0.7,0.0,0.1)
 AR1(0.8,0.0,0.1)
 AR1(0.9,0.0,0.1)

Using these pieces we can estimate the bias for each value of $T$ in a single function by simulating for the maximum value of $T$ and computing the other coefficients along the way.

In [6]:
@everywhere begin

function single_estimate(ar::AR1, Tvals::StepRange{Int}, maxT::Int)
    # Initialize storage for coefficients
    fillval = 1
    nTvals = length(Tvals)
    αhat = Array(Float64, nTvals)
    
    # Initialize variables for coefficient computation
    xiyi = 0.0
    xi = 0.0
    yi = 0.0
    xixi = 0.0

    # Initialize AR(1) state
    x = 0.0
    y = x

    for t in 1:maxT
        # One step of AR(1)
        x = step(ar, y)
        
        # Update coefficient vars
        xiyi += x*y
        xi += x
        yi += y
        xixi += x*x
        
        # Update previous value
        y = x
        if t in Tvals
            αhat[fillval] = comp_coeff(xiyi, xi, yi, xixi, t)
            fillval += 1
        end
    end
    return (αhat - ar.α)
end
    
end

The next function will run our bias function $N$ times. Notice we take advantage of the `@parallel (+)` functionality to sum up all of our biases of the estimate (remember `single_estimate` computes the coefficient for all `T` values).

In [7]:
function compute_bias(ar::AR1, Tvals::StepRange{Int}, N::Int)
    # Leave room to place bias values
    nT = length(Tvals)

    # Compute bias N times
    maxT = maximum(Tvals)
    biases = @parallel (+) for i in 1:N
        single_estimate(ar, Tvals, maxT)
    end
    
    return biases/N
end

compute_bias (generic function with 1 method)

In [17]:
function main(allar1s::Array{AR1}, Tvals::StepRange{Int}, N::Int)
    # Get information and allocate for biases
    nT = length(Tvals)
    nα = length(allar1s)
    biases = Array(Float64, nT, nα)
    
    for (iar1, ar1) in enumerate(allar1s)
        biases[:, iar1] = compute_bias(ar1, Tvals, N)
    end
    
    return biases
end
        

main (generic function with 1 method)

We can see what a large effect the better mouse trap had by benchmarking our function. It takes approximately 0.3 seconds.

In [19]:
@benchmark main(myar1, Tvals, N)

================ Benchmark Results ========================
     Time per evaluation: 311.90 ms [278.91 ms, 344.89 ms]
Proportion of time in GC: 0.02% [0.00%, 0.15%]
        Memory allocated: 1.45 mb
   Number of allocations: 19234 allocations
       Number of samples: 29
   Number of evaluations: 29
 Time spent benchmarking: 9.56 s


We can now compute and plot the biases generated by our `main` function

In [30]:
biases = main(myar1, Tvals, N)

t1 = scatter(;x=Tvals, y=biases[:, 1], name="α = $(myar1[1].α)")
t2 = scatter(;x=Tvals, y=biases[:, 2], name="α = $(myar1[2].α)")
t3 = scatter(;x=Tvals, y=biases[:, 3], name="α = $(myar1[3].α)")
t4 = scatter(;x=Tvals, y=biases[:, 4], name="α = $(myar1[4].α)")

l = Layout(;title="Downward Bias Plot",
            xaxis_title="Value of T",
            yaxis_title="Bias")

plot([t1, t2, t3, t4], l)
